In [1]:
import numpy as np
import keras2onnx
import onnxruntime
from keras.models import Sequential, load_model, Model
from keras.datasets import mnist
from metrics_face import *
import tensorflow.compat.v1 as tf
from glob import glob
import os
import facenet
from keras.preprocessing import image
from tqdm import tqdm
import keras

Using TensorFlow backend.
Can't import tf2onnx module, so the conversion on a model with any custom/lambda layer will fail!


In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
from PIL import Image
import cv2

In [ ]:
(X, y), (X_test, y_test) = mnist.load_data()
X = X[:, :, :, np.newaxis].astype('float32') / 255
X_test = X_test[:, :, :, np.newaxis].astype('float32') / 255

In [ ]:
def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(160, 160))
    return np.expand_dims(np.asarray(img),axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [ ]:
classes, nrof_classes = facenet.get_dataset("vggface2_train_2/train")
path_exp = os.path.expanduser("vggface2_train_2/train")
image_paths = [glob(os.path.join(path_exp, classes[i], "*.jpg")) for i in range(nrof_classes)]
split = int(round(nrof_classes*(1-0.99)))
X_test = paths_to_tensor([image_paths[0][0],image_paths[0][1]])

In [29]:
len(images)

30

In [28]:
classes, nrof_classes = facenet.get_dataset("../datasets/vggface2_test_2/test")
nrof_classes = 30
np.random.shuffle(classes)
path_exp = os.path.expanduser("../datasets/vggface2_test_2/test")
image_paths = [sorted(glob(os.path.join(path_exp, classes[i], "*.jpg")))[3:4] for i in range(nrof_classes)]
images = []
image_paths = np.array(image_paths).flatten().tolist()
for img_path in tqdm(image_paths):
    img = Image.open(img_path).convert('L')
    face_cascade = cv2.CascadeClassifier('/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml')
    img = np.asarray(img)
    faces_multi = face_cascade.detectMultiScale(img, 1.1, 4)
    faces = [face.astype(np.int64).tolist() for face in faces_multi]
    if len(faces) > 0:
        yes_face = faces[0]
        img = cv2.resize(img[yes_face[1]:yes_face[1]+yes_face[3],yes_face[0]:yes_face[0]+yes_face[2]],(160,160))
        images.append(img)

100%|██████████| 30/30 [00:01<00:00, 15.88it/s]


In [33]:
arcface_model = load_model('models/mnist_vgg8_arcface_5d/model_sm.hdf5', custom_objects={'ArcFace': ArcFace})

In [34]:
arcface_model1 = Model(inputs=arcface_model.input[0], outputs=arcface_model.layers[-4].output)
arcface_features1_array = []
for img in tqdm(images):
    arcface_features1 = arcface_model1.predict(img.reshape(1,160,160,1), verbose=1)
    arcface_features1_array.append(arcface_features1/np.linalg.norm(arcface_features1, axis=1, keepdims=True))

  0%|          | 0/29 [00:00<?, ?it/s]

1/1 [==============================] - 0s 277ms/step


  3%|▎         | 1/29 [00:00<00:13,  2.02it/s]

1/1 [==============================] - 0s 28ms/step


 17%|█▋        | 5/29 [00:00<00:08,  2.82it/s]

...

 76%|███████▌  | 22/29 [00:01<00:00,  9.39it/s]

1/1 [==============================] - 0s 23ms/step


 90%|████████▉ | 26/29 [00:01<00:00, 12.15it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 29/29 [00:01<00:00, 22.84it/s]


In [25]:
class_weights = [1, 0.5, 1, 10, 10]
def intermediate_model(x):
    return (keras.activations.elu((x-keras.backend.mean(x, axis=1))/keras.backend.square(keras.backend.std(x)))+\
    keras.backend.square(x-keras.backend.mean(x, axis=1))/keras.backend.std(x)) + \
keras.backend.min(x, axis=1) * keras.backend.log(keras.backend.square(x-keras.backend.mean(x, axis=1)))
intermediary_array = []
for i in tqdm(range(len(images))):
    a = np.multiply(class_weights, intermediate_model(
        tf.convert_to_tensor(tf.convert_to_tensor(arcface_features1_array[i], dtype=tf.float32))).numpy())
    if i < 10:
        print(a)
    intermediary_array.append(a)

100%|██████████| 25/25 [00:00<00:00, 110.55it/s]

[[-0.12863761  0.76236928 -0.13255814 54.14560795  4.81521785]]
[[ 0.15975255  1.33300948  0.18358797 27.40246773  1.08094156]]
[[ 3.1481061   0.43887106  2.86171556  6.6575712  30.26175976]]
[[ 0.30483395  0.13814609  2.17786789 28.44579458 20.61951399]]
[[ 2.45321512  0.27277893  2.44689369 25.40250301 37.3195672 ]]
[[ 0.08486605  1.30437589  0.12468302 35.86837769  5.72795033]]
[[ 0.14659351  2.2985568   0.21995139 20.58241606  4.15587276]]
[[ 0.12663314  0.630427    0.36587846 52.0238781  19.80523825]]
[[ 3.72222924  3.23607779  3.44074297  6.32288694 23.47439051]]
[[ 5.28120995e-03  2.34166592e-01  1.62659109e-01  6.29423046e+01
  -2.05354244e+00]]


In [ ]:
import numpy as np

In [ ]:
np.__version__

In [1]:
import pickle

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 2: ordinal not in range(128)

In [26]:
intermediary_array = np.dstack(intermediary_array)

In [ ]:
[ 0.11949151, -0.7965891 ,  0.26912665,  0.44944108,  0.27702197]
[ 0.1343845 , -0.6959256 ,  0.10052601, -0.57591254,  0.39477533]
[-0.20688158,  0.79289556, -0.17721854,  0.4884938 , -0.24183482]
[-0.23509903,  0.7068529 , -0.20450296,  0.50083554, -0.39042255]
[-0.27845502,  0.6834492 , -0.0885547 , -0.2526008 , -0.61944395]

In [27]:
gamma = arcface_model.layers[-3].weights[0]
beta = arcface_model.layers[-3].weights[1]
moving_mean = arcface_model.layers[-3].weights[2]
moving_variance = arcface_model.layers[-3].weights[3]

In [28]:
gamma.numpy(), beta.numpy(), moving_mean.numpy(), moving_variance.numpy()

(array([-0.0452649 ,  0.12484039,  0.11293622,  0.68007135,  0.11994109],
       dtype=float32),
 array([-1.3400202 , -0.639355  ,  7.2867208 , -0.77025115, -0.94090235],
       dtype=float32),
 array([-6.1528773, 11.898168 , 11.309967 , 27.529154 , -2.9971893],
       dtype=float32),
 array([ 3716.3535, 23182.342 ,  4701.81  , 16304.691 ,  8035.857 ],
       dtype=float32))

In [29]:
xNorm = (intermediary_array[0,:,:] - moving_mean.numpy().reshape(-1,1)) / np.sqrt(0.001 + moving_variance.numpy().reshape(-1,1))
data = gamma.numpy().reshape(-1,1) * xNorm + beta.numpy().reshape(-1,1)
np.round(data.T/np.linalg.norm(data.T,1),7)

array([[-0.007397 , -0.0035678,  0.0399859, -0.0034578, -0.0051191],
       [-0.0073982, -0.0035652,  0.0399887, -0.0042414, -0.0051466],
       [-0.0074104, -0.0035692,  0.040013 , -0.0048493, -0.0049318],
       [-0.0073988, -0.0035706,  0.0400068, -0.0042109, -0.0050027],
       [-0.0074076, -0.00357  ,  0.0400092, -0.0043   , -0.0048798],
       [-0.0073979, -0.0035653,  0.0399882, -0.0039934, -0.0051124],
       [-0.0073982, -0.0035609,  0.039989 , -0.0044413, -0.0051239],
       [-0.0073981, -0.0035684,  0.0399904, -0.00352  , -0.0050087],
       [-0.0074128, -0.0035566,  0.0400182, -0.0048591, -0.0049817],
       [-0.0073976, -0.0035702,  0.0399885, -0.0032   , -0.0051696],
       [-0.007397 , -0.0035715,  0.0399926, -0.0033236, -0.005036 ],
       [-0.0074146, -0.003566 ,  0.0400081, -0.0041475, -0.0051197],
       [-0.0074074, -0.0035694,  0.0400048, -0.0048247, -0.0050123],
       [-0.0073975, -0.0035708,  0.0400062, -0.003274 , -0.0051659],
       [-0.0074062, -0.0035656,  0

In [ ]:
arcface_features2_array = np.dstack(arcface_features2_array)

In [ ]:
arcface_model2 = Model(inputs=arcface_model.input[0], outputs=arcface_model.layers[-3].output)
arcface_features2_array = []
for img in tqdm(images):
    arcface_features2 = arcface_model2.predict(img.reshape(1,160,160,1), verbose=1)
    arcface_features2_array.append(arcface_features2/np.linalg.norm(arcface_features2, axis=1, keepdims=True))

In [4]:
from scipy import signal

In [3]:
p1 = [-0.176519  , -0.105028  ,  0.95071614, -0.21184155, -0.09524728]
p2 = [-0.17389667, -0.10519306,  0.93706113, -0.26787877, -0.09408321]
p3 = [-0.17534307, -0.09559851,  0.9498372 , -0.2157349 , -0.1066931 ]
p4 = [-0.17858912, -0.09505837,  0.9690808 , -0.09122588, -0.10784248]
p5 = [-0.17978127, -0.1065986 ,  0.9706176 , -0.04294081, -0.11123333]

In [ ]:
print(librosa.istft(np.dstack([p1]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p2]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p3]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p4]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p5]*5).reshape(-1,5), hop_length=1, win_length=1))

In [ ]:
from sklearn.datasets import load_iris
from pandas.plotting import andrews_curves

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# Look pretty...
matplotlib.style.use('ggplot')
# If the above line throws an error, use plt.style.use('ggplot') instead

# Load up SKLearn's Iris Dataset into a Pandas Dataframe
class_weights = [0.1, 0.5, 0.1, 1, 1]
df = pd.DataFrame(
    np.multiply(class_weights, 
      np.exp(arcface_features1_array-np.expand_dims(arcface_features1_array.mean(axis=1),1)/\
      np.expand_dims(arcface_features1_array.std(axis=1),1)) + \
    np.expand_dims(arcface_features1_array.mean(axis=1),1)
   ), columns=['col1','col2','col3','col4','col5'])
df['target_names'] = [librosa.istft(
    np.dstack([df.iloc[i,:].values]*5).reshape(-1,5), hop_length=1, win_length=1
).mean() for i in range(len(df))]
# Andrews Curves Start Here:
plt.figure()
andrews_curves(df, 'target_names')
plt.show()

In [ ]:
[-0.17978127, -0.1065986 ,  0.9706176 , -0.04294081, -0.11123333]
[-0.17668884, -0.11009184,  0.93734115, -0.26282188, -0.09474736]
[-0.1755625 , -0.05969939,  0.97241575, -0.011096  , -0.14106144]
[-0.17391324, -0.05885269,  0.9708331 ,  0.01058049, -0.15382384]
[-0.17508644, -0.07741634,  0.96141636, -0.14232956, -0.13701247]

In [ ]:
p1 = [-0.176519  , -0.105028  ,  0.95071614, -0.21184155, -0.09524728]
p2 = [-0.17389667, -0.10519306,  0.93706113, -0.26787877, -0.09408321]
p3 = [-0.17534307, -0.09559851,  0.9498372 , -0.2157349 , -0.1066931 ]
p4 = [-0.17858912, -0.09505837,  0.9690808 , -0.09122588, -0.10784248]
p5 = [-0.17978127, -0.1065986 ,  0.9706176 , -0.04294081, -0.11123333]

In [ ]:
from sklearn.datasets import load_iris
from pandas.tools.plotting import andrews_curves

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# Look pretty...
matplotlib.style.use('ggplot')
# If the above line throws an error, use plt.style.use('ggplot') instead

# Load up SKLearn's Iris Dataset into a Pandas Dataframe
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target_names'] = [data.target_names[i] for i in data.target]

In [ ]:
np.linalg.norm(arcface_features2_array, axis=1, keepdims=True)

In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imagehash.average_hash(
    Image.open("vggface2_test/test/n000029/0130_01.jpg")
)

In [ ]:
import imagehash, cv2
from PIL import Image

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_generator = datagen.flow_from_directory(
        'data/train',
        target_size=(1580, 862),
        batch_size=32)
validation_generator = datagen.flow_from_directory(
        'data/validation',
        target_size=(1580, 862),
        batch_size=32,
        class_mode='binary')


In [ ]:
fig2 = plt.figure()
ax2 = Axes3D(fig2)
for c in range(len(np.unique(y_test))):
    ax2.plot(arcface_features[y_test==c, 0], arcface_features[y_test==c, 1], arcface_features[y_test==c, 2], '.', alpha=0.1)
plt.title('ArcFace')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
print(librosa.istft(np.array([p1]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p2]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p3]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p4]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p5]*3).reshape(-1,3), hop_length=1, win_length=3))

In [ ]:
import librosa